In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
pd.set_option("display.float_format", "{:.12g}".format)
np.set_printoptions(suppress=True, precision=12) 

In [2]:
def readfile(input_filename):

     
     # Open and parse the file
     with open(input_filename, "r") as file:
          fullrun=False
          for line in file:
               
               if line.startswith("=== EVENT 1 ==="):
                    nextline = next(file)
                    #print(nextline)
                    nextline.strip()
                    start = '== TDC corrected time =' 
                    end = '=='
                    run_starttime = ((nextline.split(start))[1].split(end)[0])
                    #print(run_starttime)

               if line.startswith("=== EVENT 10000 ==="):
                    fullrun=True
                    nextline = next(file)
                    #print(nextline)
                    nextline.strip()
                    start = '== TDC corrected time =' 
                    end = '=='
                    run_endtime = ((nextline.split(start))[1].split(end)[0])
                    #print(run_endtime)

     #print(fullrun)
     if (fullrun==False):
          print("run with less than 10000 events. Stopping the execution")
          run_endtime = "0h0m0s,0.0.0ns"
          run_starttime = "0h0m0s,0.0.0ns"   
                             

     return run_starttime, run_endtime

               

In [3]:
def parse_time(time_str):
    """Extract hours, minutes, seconds, and nanoseconds from a given time string."""
    match = re.match(r"(\d+)h(\d+)m(\d+)s,(\d+)\.(\d+)\.(\d+)ns", time_str)
    if match:
        hours, minutes, seconds, ns1, ns2, ns3 = map(int, match.groups())
        nanoseconds = (hours * 3600 + minutes * 60 + seconds) * 1_000_000_000 + ns1 * 1_000_000 + ns2 * 1_000 + ns3
        return nanoseconds
    else:
        raise ValueError("Invalid time format")


In [4]:
def format_time(nanoseconds):
    """Convert nanoseconds to a human-readable h:m:s,ns format."""
    total_seconds, ns = divmod(nanoseconds, 1_000_000_000)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours}h{minutes}m{seconds}s,{ns:09d}ns"

In [5]:
def extract_rate(nanoseconds,Nevts):
    total_seconds, ns = divmod(nanoseconds, 1_000_000_000)
    return Nevts/total_seconds

In [6]:
#define path to files
file_path="/Users/bordonis/ResearchActivities/Perovskite/data/NewSiPM_KEPboard_DPNCboard/"

#list of filenames
#Cs104-A 10mV (source outside box)
#filelist = ["CERN/Cs104-A/Run_006_CERN_Cs104-A_Data_10_14_2024_Ascii.dat", 
#            "CERN/Cs104-A/Run_010_CERN_Cs104-A_Data_10_15_2024_Ascii.dat", 
#            "CERN/Cs104-A/Run_015_CERN_Cs104-A_Data_10_15_2024_Ascii.dat"]

#Cs104-A 10mV (source INSIDE box) (09.01.2025)
filelist = ["CERN/Cs104-A/Run_004_CERN_nosource_Data_1_9_2025_Ascii.dat",
            "CERN/Cs104-A/Run_005_CERN_nosource_Data_1_9_2025_Ascii.dat",
            "CERN/Cs104-A/Run_015_CERN_nosource_Data_1_9_2025_Ascii.dat"]

#Cs104-A 10mV (no source) (28.03.2025)
#filelist = ["CERN/Cs104-A/Run_025_Cs104A_Data_3_28_2025_Ascii.dat"]


#Cs104-A 15mV (source INSIDE box) (09.01.2025)
#filelist = ["CERN/Cs104-A/Run_010_CERN_nosource_Data_1_9_2025_Ascii.dat",
#            "CERN/Cs104-A/Run_016_CERN_nosource_Data_1_9_2025_Ascii.dat"]


#MAPbBr3 5mV  (17.12.2024)
#filelist = ["CERN/MAPbBr3-SC67_B/Run_000_CERN_MAPbBr3_SC67B_Data_12_4_2024_Ascii.dat",
#            "CERN/MAPbBr3-SC67_B/Run_008_CERN_Data_12_17_2024_Ascii.dat",
#            "CERN/MAPbBr3-SC67_B/Run_011_CERN_Data_12_17_2024_Ascii.dat"]

#MAPbBr3 7mV  (17.12.2024)
#filelist = ["CERN/MAPbBr3-SC67_B/Run_005_CERN_Data_12_17_2024_Ascii.dat",
#            "CERN/MAPbBr3-SC67_B/Run_006_CERN_Data_12_17_2024_Ascii.dat",
#            "CERN/MAPbBr3-SC67_B/Run_010_CERN_Data_12_17_2024_Ascii.dat"]


#filelist = ["CERN/MAPbBr3-SC67_B/Run_010_CERN_Data_12_17_2024_Ascii.dat",
#            "CERN/MAPbBr3-SC67_B/Run_001_Cs_CERN_Data_1_9_2025_Ascii.dat"]
            
#MAPbBr3 10mV  (3.10.2024) @UNIGE with Sr90
#filelist = ["MAPbBr3/SC67-B/Run_007_SC67-B_Data_10_3_2024_Ascii.dat",
#            "MAPbBr3/SC67-B/Run_006_SC67-B_Data_10_3_2024_Ascii.dat"]





In [7]:
#define output as a dictionary 

#output = {}

outputlist = []

for f in filelist:

    starttime, endtime = readfile(file_path+f)
    
    # Convert to nanoseconds
    start_ns = parse_time(starttime.strip())
    end_ns = parse_time(endtime.strip())

    # Compute difference
    diff_ns = end_ns - start_ns

    # Format the result
    time_diff_str = format_time(diff_ns)


    #output[f] = time_diff_str
    rate = extract_rate(diff_ns, 10000)

    outputlist.append((f, time_diff_str, rate))
    print("rate is [Hz]: ", rate )
    
    

rate is [Hz]:  11.26126126126126
rate is [Hz]:  41.32231404958678
rate is [Hz]:  42.5531914893617


In [8]:
pd.set_option('max_colwidth', 100)
pd.options.display.float_format = '{0:0.2f}'.format
df_out = pd.DataFrame(outputlist, columns=['File', 'Time duration', 'Rate'])
df_out.head()

,File,Time duration,Rate
0,CERN/Cs104-A/Run_004_CERN_nosource_Data_1_9_2025_Ascii.dat,"0h14m48s,339597584ns",11.26
1,CERN/Cs104-A/Run_005_CERN_nosource_Data_1_9_2025_Ascii.dat,"0h4m2s,082170709ns",41.32
2,CERN/Cs104-A/Run_015_CERN_nosource_Data_1_9_2025_Ascii.dat,"0h3m55s,809173236ns",42.55


In [9]:
#some manual computation for runs with less than 10k events (script to be improved)

run_start_nosource = "12h55m49s,159.990.455ns"
run_stop_nosource = "15h30m09s,436.246.149ns"
Nevts_nosource=7005

run_start_Am241="15h32m45s,279.525.690ns"
run_stop_Am241="16h14m34s,930.905.916ns"
Nevts_Am241=2053

run_start_Cs137="16h54m12s,389.705.530ns"
run_stop_Cs137="17h23m38s,669.663.064ns"
Nevts_Cs137=10000

In [ ]:
def extract_rate(nanoseconds,Nevts):
    total_seconds, ns = divmod(nanoseconds, 1_000_000_000)
    return Nevts/total_seconds

In [ ]:
# Convert to nanoseconds
start_ns = parse_time(run_start_Am241.strip())
end_ns = parse_time(run_stop_Am241.strip())

# Compute difference
diff_ns = end_ns - start_ns

# Format the result
time_diff_str = format_time(diff_ns)

print(time_diff_str)
rate = extract_rate(diff_ns,Nevts_Am241)
print("rate is [Hz]: ", rate)